In [ ]:
import deepxde as dde
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from naca4digit_airfoil import Naca4DigitAirfoil
from navier_stokes import NavierStokesPDEs
import utils
import pandas as pd
from scipy.interpolate import griddata

In [ ]:
cfd_df = pd.read_csv('cfd_data.csv', delimiter=',')

In [ ]:
x_min_init = -0.25
x_max_init = 0.75
y_min_init = -0.25
y_max_init = 0.25

x_min = -0.05
x_max = 0.15
y_min = -0.05
y_max = 0.05

In [ ]:
def set_domain_lims(ax):
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)

def set_zoomed_domain_lims(ax):
    ax.set_xlim(x_min/2, x_max/2)
    ax.set_ylim(y_min/2, y_max/2)

def set_labels(ax):
    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')

In [ ]:
dde.config.set_random_seed(48)
dde.config.set_default_float('float64')
dde.config.set_parallel_scaling('strong')

In [ ]:
rho  = 1.225
# mu   = 1.789e-5
mu   = 0.0002
# u_in  = 15
u_inlet  = 0.75
L = 0.05

Re = rho * u_inlet * L / mu

In [ ]:
rho, mu, u_inlet, L, Re

In [ ]:
airfoil = Naca4DigitAirfoil(c=L, M=2, P=4, T=12, a=0, offset_x=0, offset_y=0)

In [ ]:
# Geometry defintion
farfield = dde.geometry.Rectangle([x_min, y_min], [x_max, y_max])
airfoil_geom  = dde.geometry.Polygon(airfoil.get_boundary_points(250))
geom     = dde.geometry.CSGDifference(farfield, airfoil_geom)

inner_rec  = dde.geometry.Rectangle([-0.01, -0.01], [0.01, 0.01])
inner_dom  = dde.geometry.CSGDifference(inner_rec, airfoil_geom)

mid_rec  = dde.geometry.Rectangle([-0.03, -0.02], [0.075, 0.02])
mid_dom  = dde.geometry.CSGDifference(mid_rec, inner_rec)
mid_dom  = dde.geometry.CSGDifference(mid_dom, airfoil_geom)

outer_dom  = dde.geometry.CSGDifference(farfield, inner_rec)
outer_dom  = dde.geometry.CSGDifference(outer_dom, mid_rec)
outer_dom  = dde.geometry.CSGDifference(outer_dom, airfoil_geom)

In [ ]:
Nf1 = 2**14 # = 16384
Nf2 = 2**14 # = 16384
Nf3 = 2**16 # = 65536
Nb  = 2**11 # = 2048
Ns  = 250

random = "Sobol"
inner_points = inner_dom.random_points(Nf1, random=random)
mid_points = mid_dom.random_points(Nf2, random=random)
outer_points = outer_dom.random_points(Nf3, random=random)

farfield_points = farfield.random_boundary_points(Nb, random=random)
airfoil_points  = airfoil.get_boundary_points(Ns)

static_points = np.append(inner_points, mid_points, axis = 0)
static_points = np.append(static_points, airfoil_points, axis = 0)


In [ ]:
fig, ax = plt.subplots(figsize=(18, 10))
set_labels(ax)
ax.set_title('Sampled Training Points')
ax.scatter(inner_points[:, 0], inner_points[:, 1], s = 0.1, c = '#76c68f', label = 'Inner domain collocation points ($N_{f_1}$)')
ax.scatter(mid_points[:, 0], mid_points[:, 1], s = 0.1, c = '#3498db', label = 'Mid domain collocation points ($N_{f_2}$)')
ax.scatter(outer_points[:, 0], outer_points[:, 1], s = 0.1, c = '#d4ac0d', label = 'Outer domain collocation points ($N_{f_3})$')
ax.scatter(airfoil_points[:, 0], airfoil_points[:, 1], s = 0.1, c = '#f1948a', label = 'Airfoil boundary points ($N_s$)')
ax.scatter(farfield_points[:, 0], farfield_points[:, 1], s = 0.1, c='#9b59b6', label = 'CFD data points ($N_u$)')
# ax.fill(airfoil_points[:, 0], airfoil_points[:, 1])
ax.grid(False)
legend = ax.legend(loc = 'upper right')
for legend_handle in legend.legendHandles:
    legend_handle._sizes = [30]  # Adjusting the size of the markers in the legend
plt.savefig('hybrid_sampling1.png', dpi=300)
plt.show()

In [ ]:
# X, Y = np.meshgrid(np.linspace(cfd_df['x'].min(), cfd_df['x'].max(), 1000),
#                              np.linspace(cfd_df['y'].min(), cfd_df['y'].max(), 1000))

# U = griddata((cfd_df['x'], cfd_df['y']), cfd_df['u'], (X, Y), method='linear')

# fig, ax = plt.subplots(figsize=(10, 6))

# clev = np.arange(U.min(), U.max(), 0.01)
# cp = ax.contourf(X, Y, U, clev, cmap='jet')
# fig.colorbar(cp, ax=ax)
# ax.set_xlabel('X')
# ax.set_ylabel('Y')
# ax.set_title('Velocity Field - X Component')
# airfoil.plot(ax)
# ax.set_xlim(x_min, x_max)
# ax.set_ylim(y_min, y_max)
# plt.show()

In [ ]:
# V = griddata((cfd_df['x'], cfd_df['y']), cfd_df['v'], (X, Y), method='linear')

# fig, ax = plt.subplots(figsize=(10, 6))

# clev = np.arange(V.min(), V.max(), 0.01)
# cp = ax.contourf(X, Y, V, clev, cmap='jet')
# fig.colorbar(cp, ax=ax)
# ax.set_xlabel('X')
# ax.set_ylabel('Y')
# ax.set_title('Velocity Field - Y Component')
# airfoil.plot(ax)
# ax.set_xlim(x_min, x_max)
# ax.set_ylim(y_min, y_max)
# plt.show()

In [ ]:
# P = griddata((cfd_df['x'], cfd_df['y']), cfd_df['p'], (X, Y), method='linear')

# fig, ax = plt.subplots(figsize=(10, 6))

# clev = np.arange(P.min(), P.max(), 1)
# cp = ax.contourf(X, Y, P, clev, cmap='jet')
# fig.colorbar(cp, ax=ax)
# ax.set_xlabel('X')
# ax.set_ylabel('Y')
# ax.set_title('Pressure Field')
# airfoil.plot(ax)
# ax.set_xlim(x_min, x_max)
# ax.set_ylim(y_min, y_max)
# plt.show()

In [ ]:
x_data = farfield_points[:, 0]
y_data = farfield_points[:, 1]

grid_points = cfd_df[['x', 'y']].values
u_values = cfd_df['u'].values
v_values = cfd_df['v'].values
p_values = cfd_df['p'].values

u_interp = griddata(grid_points, u_values, (x_data, y_data), method='linear')
v_interp = griddata(grid_points, v_values, (x_data, y_data), method='linear')
p_interp = griddata(grid_points, p_values, (x_data, y_data), method='linear')

uvp_data = np.vstack((u_interp, v_interp, p_interp)).T
u_data = np.array([uvp_data[i][0] for i in range(len(uvp_data))])
v_data = np.array([uvp_data[i][1] for i in range(len(uvp_data))])
p_data = np.array([uvp_data[i][2] for i in range(len(uvp_data))])

In [ ]:
fig, ax = plt.subplots(figsize=(18, 10))
plt.axis('equal')
scatter = plt.scatter(x_data, y_data, c=u_data, s=2, cmap='jet')
set_labels(ax)
plt.colorbar(scatter, ax=ax, label='Velocity')
plt.title('CFD Solver Boundary Data, Velocity Field - X Component')
airfoil.plot(ax)
plt.xlim([x_min - 0.01, x_max + 0.01])
plt.ylim([y_min - 0.01, y_max + 0.01])
ax.grid(False)
plt.savefig('u_boundary_cfd.png', dpi=300)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(18, 10))
plt.axis('equal')
scatter = plt.scatter(x_data, y_data, c=v_data, s=2, cmap='jet')
set_labels(ax)
plt.colorbar(scatter, ax=ax, label='Velocity')
plt.title('CFD Solver Boundary Data, Velocity Field - Y Component')
airfoil.plot(ax)
plt.xlim([x_min - 0.01, x_max + 0.01])
plt.ylim([y_min - 0.01, y_max + 0.01])
ax.grid(False)
plt.savefig('v_boundary_cfd.png', dpi=300)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(18, 10))
plt.axis('equal')
scatter = plt.scatter(x_data, y_data, c=p_data, s=2, cmap='jet')
set_labels(ax)
plt.colorbar(scatter, ax=ax, label='Velocity')
plt.title('CFD Solver Boundary Data, Pressure Field')
airfoil.plot(ax)
plt.xlim([x_min - 0.01, x_max + 0.01])
plt.ylim([y_min - 0.01, y_max + 0.01])
ax.grid(False)
plt.savefig('p_boundary_cfd.png', dpi=300)
plt.show()

In [ ]:
navier_stokes_pdes = NavierStokesPDEs(rho=rho, mu=mu,
                                      xmin=x_min, xmax=x_max, ymin=y_min, ymax=y_max,
                                      airfoil_geom=airfoil_geom, geom=geom,
                                      x_data=x_data, y_data=y_data, u_data=u_data, v_data=v_data, p_data=p_data)

pdes_fun = navier_stokes_pdes.get_pdes
bcs = navier_stokes_pdes.get_bcs_with_data()

In [ ]:
# Problem setup
data = dde.data.PDE(geom, pdes_fun, bcs, num_domain = Nf3, num_boundary = 0, num_test = 1000, anchors = static_points, train_distribution = "Hammersley")

In [ ]:
fig, ax = plt.subplots(figsize=(18, 10))
set_labels(ax)
ax.set_title('Sampled Training Points')
ax.scatter(data.train_x[:,0], data.train_x[:,1], s = 0.1)
ax.grid(False)
plt.savefig('hybrid_sampling2.png', dpi=300)
plt.show()

In [ ]:
# Neural network definition
layer_size  = [2] + [50] * 10 + [5]
activation  = 'tanh'
initializer = 'Glorot uniform'

net = dde.nn.FNN(layer_size, activation, initializer)

# Model definition
model = dde.Model(data, net)

In [ ]:
name_generator = utils.NameGenerator()
model_name = name_generator.generate_name()
model_name

In [ ]:
model.compile(optimizer = 'adam', lr = 5e-4, loss_weights=[1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2])

In [ ]:
resampler = dde.callbacks.PDEPointResampler(period = 100, pde_points=True, bc_points=False)

In [ ]:
try:
  losshistory, train_state = model.train(epochs = 1000, display_every = 50, callbacks=[resampler], model_save_path = './' + model_name + '/')
  dde.saveplot(losshistory, train_state, issave = True, isplot = True)

except KeyboardInterrupt:
  print(f"Training stopped by user.")
  print("=======================================================")

In [ ]:
# model_name = 'AtmosElegantGalaxy375'
# model.restore('./out/AtmosElegantGalaxy375/-20000.ckpt.index')

In [ ]:
# Plotting tool: thanks to @q769855234 code snippet
dx = 0.025
dy = 0.025
x = np.arange(x_min, x_max + dy, dx)
y = np.arange(y_min, y_max + dy, dy)

X = np.zeros((len(x)*len(y), 2))
xs = np.vstack((x,)*len(y)).reshape(-1)
ys = np.vstack((y,)*len(x)).T.reshape(-1)
X[:, 0] = xs
X[:, 1] = ys

In [ ]:
# Model predictions generation
u = model.predict(X, operator = navier_stokes_pdes.getU)
v = model.predict(X, operator = navier_stokes_pdes.getV)
p = model.predict(X, operator = navier_stokes_pdes.getP)

for i in range(len(X)):
   if airfoil_geom.inside(np.array([X[i]]))[0]:
       u[i] = 0.0
       v[i] = 0.0

u = u.reshape(len(y), len(x))
v = v.reshape(len(y), len(x))
p = p.reshape(len(y), len(x))

airfoil_plot = airfoil.get_boundary_points(150)

fig1, ax1 = plt.subplots(figsize = (16, 9))
ax1.streamplot(x, y, u, v, density = 1.5)
clev = np.arange(u.min(), u.max(), 0.001)
cnt1 = ax1.contourf(x, y, u, clev, cmap = plt.cm.jet, extend='both')
plt.axis('equal')
plt.fill(airfoil_plot[:, 0], airfoil_plot[:, 1])
fig1.colorbar(cnt1)
plt.savefig('./u.png')

fig2, ax2 = plt.subplots(figsize = (16, 9))
ax2.streamplot(x, y, u, v, density = 1.5)
clev = np.arange(v.min(), v.max(), 0.00001)
cnt2 = ax2.contourf(x, y, v, clev, cmap = plt.cm.jet, extend='both')
plt.axis('equal')
plt.fill(airfoil_plot[:, 0], airfoil_plot[:, 1])
fig2.colorbar(cnt2)
plt.savefig('./v.png')

fig3, ax3 = plt.subplots(figsize = (16, 9))
clev = np.arange(p.min(), p.max(), 0.0001)
cnt3 = ax3.contourf(x, y, p, clev, cmap = plt.cm.jet, extend='both')
plt.axis('equal')
plt.fill(airfoil_plot[:, 0], airfoil_plot[:, 1])
fig3.colorbar(cnt3)
plt.savefig('./p.png')